In [1]:
import os
import pandas as pd
import shutil
import re
import pprint
import math

# INPUTS

In [2]:
#import all the sheets from the inputs excel file
inputs_sheetDict = pd.read_excel('Inputs.xlsx', sheet_name=None)

In [3]:
# Access individual sheets as dataframes from the dictionary
CUP_inputs_df = inputs_sheetDict['Sheet1']
building_inputs_df = inputs_sheetDict['Sheet2']
TES_inputs_df = inputs_sheetDict['Sheet3']
hotTank_schedule1_inputs_df = inputs_sheetDict['Sheet4']
coldTank_schedule2_inputs_df = inputs_sheetDict['Sheet5']
hotWater_schedule3_inputs_df = inputs_sheetDict['Sheet6']
domesticWater_schedule4_inputs_df = inputs_sheetDict['Sheet7']
chilledCTOcean_schedule5_inputs_df = inputs_sheetDict['Sheet8']
groundReturn_schedule6_inputs_df = inputs_sheetDict['Sheet9']
groundReturn_schedule6_inputs_df.head(15)


,Schedules,Ground Return Temps,Schedules Month,Ht Rej - LWT*,Ht Rej - EWT,Ht Ex - LWT*,Ht Ex - EWT
0,Schedules,Ground Return Temps,1,70,NaN,60,NaN
1,Schedules,Ground Return Temps,2,65,NaN,55,NaN
2,Schedules,Ground Return Temps,3,60,NaN,50,NaN
3,Schedules,Ground Return Temps,4,62,NaN,52,NaN
4,Schedules,Ground Return Temps,5,65,NaN,55,NaN
5,Schedules,Ground Return Temps,6,70,NaN,60,NaN
6,Schedules,Ground Return Temps,7,75,NaN,65,NaN
7,Schedules,Ground Return Temps,8,80,NaN,70,NaN
8,Schedules,Ground Return Temps,9,84,NaN,74,NaN
9,Schedules,Ground Return Temps,10,85,NaN,75,NaN


In [4]:
## Adding formulas for groundReturn_schedule6_inputs_df
# Apply the formula to calculate 'Ht Ex - EWT' column
groundReturn_schedule6_inputs_df['Ht Ex - EWT'] = groundReturn_schedule6_inputs_df['Ht Ex - LWT*'].apply(lambda x: x-10 if not pd.isna(x) else float('nan'))
# Apply the formula to calculate 'Ht Ex - EWT' column
groundReturn_schedule6_inputs_df['Ht Rej - EWT'] = groundReturn_schedule6_inputs_df['Ht Rej - LWT*'].apply(lambda x: 10 + x if not pd.isna(x) else float('nan'))
groundReturn_schedule6_inputs_df.head(15)

,Schedules,Ground Return Temps,Schedules Month,Ht Rej - LWT*,Ht Rej - EWT,Ht Ex - LWT*,Ht Ex - EWT
0,Schedules,Ground Return Temps,1,70,80,60,50
1,Schedules,Ground Return Temps,2,65,75,55,45
2,Schedules,Ground Return Temps,3,60,70,50,40
3,Schedules,Ground Return Temps,4,62,72,52,42
4,Schedules,Ground Return Temps,5,65,75,55,45
5,Schedules,Ground Return Temps,6,70,80,60,50
6,Schedules,Ground Return Temps,7,75,85,65,55
7,Schedules,Ground Return Temps,8,80,90,70,60
8,Schedules,Ground Return Temps,9,84,94,74,64
9,Schedules,Ground Return Temps,10,85,95,75,65


In [23]:
##Adding formulas for Tank_schedule1_inputs_df
# Sum the values of the columns from index 1 to 23 (corresponding to hours 0 to 23)
#hot tank
hotTank_schedule1_inputs_df['Total'] = hotTank_schedule1_inputs_df.loc[:, 0:23].sum(axis=1)
hotTank_schedule1_inputs_df.head(15)
#cold tank
coldTank_schedule2_inputs_df['Total'] = coldTank_schedule2_inputs_df.loc[:, 0:23].sum(axis=1)
coldTank_schedule2_inputs_df.head(15)

,Schedules,Load Shift Control,Month,0,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,Total
0,Schedules,Load Shift Control - Cold Tank by month,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
1,Schedules,Load Shift Control - Cold Tank by month,2,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
2,Schedules,Load Shift Control - Cold Tank by month,3,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
3,Schedules,Load Shift Control - Cold Tank by month,4,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
4,Schedules,Load Shift Control - Cold Tank by month,5,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
5,Schedules,Load Shift Control - Cold Tank by month,6,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
6,Schedules,Load Shift Control - Cold Tank by month,7,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
7,Schedules,Load Shift Control - Cold Tank by month,8,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
8,Schedules,Load Shift Control - Cold Tank by month,9,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
9,Schedules,Load Shift Control - Cold Tank by month,10,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9


In [5]:
# the function to changes input dataframe to dictionaries(single value input)

def createDict(df):
    
    dictName = df.iloc[:,0].unique()
    
    # Initialize an empty dictionary to store the data
    data_dict = {}
    
    # Filter out rows with NaN category (if any)
    df = df.dropna(subset=[df.columns[1]])
    
    # Group by category and create nested dictionaries using pandas groupby
    grouped = df.groupby(df.columns[1])
    
    for category, group_df in grouped:
        data_dict[category] = dict(zip(group_df.iloc[:, 2], group_df.iloc[:, 3]))
        
    
    return data_dict  
        
#single input value dicts       
CUP_inputs = createDict(CUP_inputs_df)   
building_inputs = createDict(building_inputs_df) 
TES_inputs = createDict(TES_inputs_df) 
domesticWater_sch4_inputs = createDict(domesticWater_schedule4_inputs_df)
chilledCTOcean_sch5_inputs = createDict(chilledCTOcean_schedule5_inputs_df)

pprint.pprint(CUP_inputs)

{'Electric Boilers': {'Boiler Efficiency': 0.99,
                      'Hot Water Setpoint': 170.0,
                      'Max Capacity': 10000000.0},
 'Electric Chillers': {'Capacity for Min Lift': nan,
                       'Max Capacity': 60000000.0,
                       'Max Lift': 42.0,
                       'Min Lift': 15.0},
 'Heat Pumps': {'Air-Source + Water Source %': 0.5,
                'Cooling Capacity': 30000000.0,
                'Heating Capacity': 40000000.0,
                'Water-Source Only %': 0.5},
 'Heat Source/Sink': {'Cooling Tower Capacity': 0.0,
                      'Delta T (for Cent. Chillers; CT & Ocean Only)': 15.0,
                      'Delta T (for HPs; CT & Ocean Only)': 10.0,
                      'Ground-source Capacity': 100000000.0,
                      'Ocean Capacity': 0.0},
 'Setpoints': {'Chilled Water Return Losses': 0.0,
               'Chilled Water Supply': nan,
               'Chilled Water Supply Losses': 2.0,
               'Hot 

In [6]:
# the function to changes input dataframe to dictionaries(multiple value input)

def createDict_multipleValues(df):
    
  
    # Initialize an empty dictionary to store the data
    data_dict = {}
    
    # Filter out rows with NaN category (if any)
    df = df.dropna(subset=[df.columns[1]])
    
    # Group by category and create nested dictionaries using pandas groupby
    grouped = df.groupby(df.columns[1])
    
    for category, group_df in grouped:
        # Initialize a nested dictionary for the current category
        category_dict = {}
        
        for index,row in group_df.iloc[:,2:].iterrows():
            month = row[ group_df.columns[2]]
            if month not in category_dict:
                     category_dict[month] = {}
            for hour_column,value in row.items():
                # Skip if the column name is the month column or NaN value
                if hour_column == group_df.columns[2]:
                    continue
                
                # Extract the hour (column name as integer)
                hour = hour_column
                
#                 # Create a compound key using month and hour
#                 key = (month, hour)
                
                category_dict[month][hour] = value
#                 print(key)
    # Add the category dictionary to the main data dictionary
    data_dict[category] = category_dict
    return data_dict  

hotTank_sch1_inputs = createDict_multipleValues(hotTank_schedule1_inputs_df)
coldTank_sch2_inputs = createDict_multipleValues(coldTank_schedule2_inputs_df)
hotWater_sch3_inputs = createDict_multipleValues(hotWater_schedule3_inputs_df)
groundReturn_sch6_inputs = createDict_multipleValues(groundReturn_schedule6_inputs_df)
pprint.pprint(groundReturn_sch6_inputs)

{'Ground Return Temps': {1: {'Ht Ex - EWT': 50,
                             'Ht Ex - LWT*': 60,
                             'Ht Rej - EWT': 80,
                             'Ht Rej - LWT*': 70},
                         2: {'Ht Ex - EWT': 45,
                             'Ht Ex - LWT*': 55,
                             'Ht Rej - EWT': 75,
                             'Ht Rej - LWT*': 65},
                         3: {'Ht Ex - EWT': 40,
                             'Ht Ex - LWT*': 50,
                             'Ht Rej - EWT': 70,
                             'Ht Rej - LWT*': 60},
                         4: {'Ht Ex - EWT': 42,
                             'Ht Ex - LWT*': 52,
                             'Ht Rej - EWT': 72,
                             'Ht Rej - LWT*': 62},
                         5: {'Ht Ex - EWT': 45,
                             'Ht Ex - LWT*': 55,
                             'Ht Rej - EWT': 75,
                             'Ht Rej - LWT*': 65},
               

In [7]:
# # the function to changes input dataframe to dictionaries(single value input)

# def createDict(df):
    
#     dictName = df.iloc[:,0].unique()
    
#     # Initialize an empty dictionary to store the data
#     dictName = {}
    
#     # Iterate over rows in the DataFrame to add all values in the dictionary
#     for index,row in df.iterrows():
        
#         if index == 0:
#             continue  # Skip the first row (header)
    
#         category = row[df.columns[1]]
#         itemName = row[df.columns[2]]
#         value = row[df.columns[3]]
    
#         #skip row incase of no value 
#         if pd.isna(category):
#             continue
#         # Create a dictionary for the current category if it doesn't exist
#         if category not in dictName:
#             dictName[category] = {}

#         #add the value to that item in that category in that dataframe(double nested dict)
#         dictName[category][itemName] = value
        
#         category = None
#         itemName = None
#         value = None
    
#     return dictName     
        
# #single input value dicts       
# CUP_inputs = createDict(CUP_inputs_df)   
# building_inputs = createDict(building_inputs_df) 
# TES_inputs = createDict(TES_inputs_df) 
# schedule4_inputs = createDict(schedule4_inputs_df)
# schedule5_inputs = createDict(schedule5_inputs_df)

# pprint.pprint(CUP_inputs)

# STATIC INPUTS

In [8]:
#import Weather data 
weather_df = pd.read_excel('Weather data.xlsx')
weather_df.head()

,\n\n\nDate,HH:MM,Day,Month,Dry Bulb Temperature {C},Dew Point Temperature {C},Relative Humidity {%},Dry Bulb Temp (°F),Dew Point Temp (°F),Wet Bulb Temp (°F)
0,2005-01-01,01:00:00,1,1,12.3,8.2,76,54.14,46.76,49.501190
1,2005-01-01,02:00:00,1,1,13.2,7.3,68,55.76,45.14,49.524185
2,2005-01-01,03:00:00,1,1,12.2,8.2,77,53.96,46.76,49.515631
3,2005-01-01,04:00:00,1,1,12.2,7.7,74,53.96,45.86,48.960694
4,2005-01-01,05:00:00,1,1,12.2,7.3,72,53.96,45.14,48.592221


In [9]:
#import Ocean data 
ocean_df = pd.read_excel('Ocean temp.xlsx')
ocean_df.head()

,\n\n\nYear,Month,Day,Surface Temp (°C),Surface Temp (°F)
0,2021,1,1,NaN,55.94
1,2021,1,2,13.3,55.94
2,2021,1,3,13.3,55.94
3,2021,1,4,13.3,55.94
4,2021,1,5,13.4,56.12


In [10]:
#import heat pump data 
heatPump_df = pd.read_excel('CO2 heat pump details.xlsx')
heatPump_df.head()

,CO2 Heat Pump\nLeaving Hot Water (°F),Entering Hot Water (°F),Max Heating Output (Btu/h),HW Flow Rate (gpm),Entering Cold Water (°F),Leaving Cold Water (°F),Max Cooling Output (Btu/h),CHW Flow Rate (gpm),Electrical Input,Cooling COP,Heating COP,Combined COP
0,170,80,146.0,38.9,65,44,104.9,167.9,119.8,3.1,4.3,7.4
1,180,80,146.0,35.0,65,44,104.9,167.9,119.8,3.1,4.3,7.4
2,170,80,142.7,38.1,56,44,101.5,203.0,122.9,2.9,4.1,7.0
3,180,80,142.7,34.3,56,44,101.5,203.0,122.9,2.9,4.1,7.0
4,170,80,140.1,37.4,46,44,98.8,296.5,125.4,2.8,3.9,6.7


In [11]:
#import building usage data
buildingUsage_df = pd.read_excel('Building usage data.xlsx')
# Sum the loads
totalSpaceHeating = buildingUsage_df['Heating Load (Btu/h)'].sum()
totalDHW = buildingUsage_df['DHW Load (Btu/h)'].sum()
totalHeating = buildingUsage_df['Total Heating Load (Btu/h)'].sum()
totalLoad = buildingUsage_df['Total Load (Btu/h)'].sum()
buildingUsage_df.head()



,Date and Time,Heating Load (Btu/h),DHW Load (Btu/h),Cooling Load (Btu/h),Total Heating Load (Btu/h),Total Load (Btu/h)
0,40179.000000,6.158529e+06,1.913677e+06,48.929039,8.072206e+06,8.072255e+06
1,40179.041667,4.513664e+06,1.618239e+06,349.285328,6.131903e+06,6.132252e+06
2,40179.083333,5.149001e+06,1.734384e+06,349.285328,6.883384e+06,6.883734e+06
3,40179.125000,5.668696e+06,1.582614e+06,349.285328,7.251309e+06,7.251658e+06
4,40179.166667,5.888233e+06,2.090599e+06,349.285328,7.978832e+06,7.979181e+06


In [12]:
#calculating the inputs
# CUP setpoints
maxValue_hotWater = hotWater_schedule3_inputs_df["CUP"].max()
CUP_inputs["Setpoints"]["Hot Water Supply"] = maxValue_hotWater
maxValue_chilledWater = chilledCTOcean_schedule5_inputs_df.loc[chilledCTOcean_schedule5_inputs_df["Schedules Category"]=="Chilled Water Setpoint","CUP"].max()
CUP_inputs["Setpoints"]["Chilled Water Supply"] = maxValue_chilledWater

#CUP electric chillers
CUP_inputs["Electric Chillers"]["Capacity for Min Lift"] = CUP_inputs["Electric Chillers"]["Max Capacity"]*0.3

#building Chilled water
maxValue_cooling = buildingUsage_df['Cooling Load (Btu/h)'].max()
building_inputs["Chilled Water"]["Max Load"]= maxValue_cooling

#building DHW
maxValue_DHW = buildingUsage_df['DHW Load (Btu/h)'].max()
building_inputs["Domestic Hot Water"]["Max Load"]= maxValue_DHW
building_inputs["Domestic Hot Water"]["Load @ Min Approach"]= maxValue_DHW*0.26

#building HHW
maxValue_HHW = buildingUsage_df['Heating Load (Btu/h)'].max()
building_inputs["Heating Hot Water"]["Max Load"]= maxValue_HHW + building_inputs["Domestic Hot Water"]["Max Load"]

#TES Tank Tank Properties
CUFtoGAL = TES_inputs["Conversion Rates"]["CUF to GAL"]
hot_radius = TES_inputs["Tank Properties"]["TES Hot Diameter"]/2
hot_height = TES_inputs["Tank Properties"]["TES Hot Height"]
TES_inputs["Tank Properties"]["TES Hot Tank Volume"]= math.pi * (hot_radius ** 2) * hot_height * CUFtoGAL
cold_radius = TES_inputs["Tank Properties"]["TES Cold Diameter"]/2
cold_height = TES_inputs["Tank Properties"]["TES Cold Height"]
TES_inputs["Tank Properties"]["TES Cold Tank Volume"]= math.pi * (cold_radius ** 2) * cold_height * CUFtoGAL

#TES Initial Conditions
# TES Hot Tank Upper Tank Volume ----- Check the condition and calculate the result accordingly
if TES_inputs["Tank Properties"]["TES Hot Tank Volume"] == 0:
    result = 0
else:
    hot_volume1 = TES_inputs["Tank Properties"]["TES Hot Tank Volume"]
    hot_volume2 =TES_inputs["Initial Conditions"]["TES Hot Thermocline thickness"] * math.pi * (hot_radius ** 2)* CUFtoGAL   # Volume of second part of the cylinder
    difference = hot_volume1 - hot_volume2           # Difference between the volumes
    TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Volume"] = difference * TES_inputs["Initial Conditions"]["Starting TES Hot Charge"]          
TESHotUpperTankVolume = TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Volume"]
    
# TES Hot Thermocline volume
if TES_inputs["Tank Properties"]["TES Hot Tank Volume"] == 0:
    result = 0
else:
    TES_inputs["Initial Conditions"]["TES Hot Thermocline volume"] =  hot_volume2

    
#TES Hot Tank Lower Tank Volume    
TES_inputs["Initial Conditions"]["TES Hot Tank Lower Tank Volume"] =  hot_volume1 - hot_volume2 - TESHotUpperTankVolume    
    
# TES Cold Tank Lower Tank Volume ----- Check the condition and calculate the result accordingly
if TES_inputs["Tank Properties"]["TES Cold Tank Volume"] == 0:
    result = 0
else:
    cold_volume1 =TES_inputs["Tank Properties"]["TES Cold Tank Volume"]
    cold_volume2 =TES_inputs["Initial Conditions"]["TES Cold Thermocline thickness"] * math.pi * (cold_radius ** 2)* CUFtoGAL   # Volume of second part of the cylinder
    difference = cold_volume1 - cold_volume2           # Difference between the volumes
    TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Volume"] = difference * TES_inputs["Initial Conditions"]["Starting TES Cold Charge"]           
TESColdLowerTankVolume =  TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Volume"]
# TES cold Thermocline volume
if TES_inputs["Tank Properties"]["TES Cold Tank Volume"] == 0:
    result = 0
else:
    TES_inputs["Initial Conditions"]["TES Cold Thermocline volume"] =  cold_volume2 
#TES Hot Tank Lower Tank Volume    
TES_inputs["Initial Conditions"]["TES Cold Tank Upper Tank Volume"] =  cold_volume1 - cold_volume2 - TESColdLowerTankVolume

#Tank Charging Characteristics
#TES Hot Capacity
GALtoLbs = TES_inputs["Conversion Rates"]["GAL to Lbs"]
upperHotTemp = TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Temp"]
lowerHotTemp = TES_inputs["Initial Conditions"]["TES Hot Tank Lower Tank Temp"]
TES_inputs["Tank Charging Characteristics"]["TES Hot Capacity"] = GALtoLbs*(upperHotTemp-lowerHotTemp)*hot_volume1/1000
#TES Hot Max Flow 
tankVolumeMoved = TES_inputs["Tank Charging Characteristics"]["TES Hot Max % of Total Tank Volume moved in 1 hr"]
TES_inputs["Tank Charging Characteristics"]["TES Hot Max Flow"] = math.ceil((hot_volume1 * tankVolumeMoved) / 60 / 5) * 5
#TES Hot Max Flow 
hot_tankVolumeMoved = TES_inputs["Tank Charging Characteristics"]["TES Hot Max % of Total Tank Volume moved in 1 hr"]
TES_inputs["Tank Charging Characteristics"]["TES Hot Max Flow"] = math.ceil((hot_volume1 * hot_tankVolumeMoved) / 60 / 5) * 5

#TES Cold Capacity
GALtoLbs = TES_inputs["Conversion Rates"]["GAL to Lbs"]
upperColdTemp = TES_inputs["Initial Conditions"]["TES Cold Tank Upper Tank Temp"]
lowerColdTemp = TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Temp"]
TES_inputs["Tank Charging Characteristics"]["TES Cold Capacity"] = GALtoLbs*(upperColdTemp-lowerColdTemp)*cold_volume1/1000
#TES Cold Max Flow 
cold_tankVolumeMoved = TES_inputs["Tank Charging Characteristics"]["TES Cold Max % of Total Tank Volume moved in 1 hr"]
TES_inputs["Tank Charging Characteristics"]["TES Cold Max Flow"] = math.ceil((cold_volume1 * cold_tankVolumeMoved) / 60 / 5) * 5




# Building heating hot water 
pprint.pprint(TES_inputs)


{'Conversion Rates': {'CUF to GAL': 7.48, 'GAL to Lbs': 8.34},
 'Initial Conditions': {'Starting TES Cold Charge': 0.99,
                        'Starting TES Hot Charge': 0.99,
                        'TES Cold Tank Lower Tank Temp': 44.0,
                        'TES Cold Tank Lower Tank Volume': 1214387.1641385558,
                        'TES Cold Tank Upper Tank Temp': 60.0,
                        'TES Cold Tank Upper Tank Volume': 12266.53701150068,
                        'TES Cold Thermocline thickness': 2.0,
                        'TES Cold Thermocline volume': 42298.40348793298,
                        'TES Hot Tank Lower Tank Temp': 80.0,
                        'TES Hot Tank Lower Tank Volume': 12266.53701150068,
                        'TES Hot Tank Upper Tank Temp': 170.0,
                        'TES Hot Tank Upper Tank Volume': 1214387.1641385558,
                        'TES Hot Thermocline thickness': 2.0,
                        'TES Hot Thermocline volume': 42298.

In [13]:

# for equipment in CUPunitCost["Equipment name"]:
#     size = CUPequipmentsizes.loc["Equipment name" == equipment,"Size"]
#     unitCost = CUPunitcost.loc["Equipment name" == equipment,"Unit Cost"]
#     CUPequipmentfirstcost["First cost"] = size*unitCost


In [14]:
%cd "C:\\Users\\nikita.khatwani\\Documents\\UCSB\\District sytem - Engineering model\\District system - Engineering model"

[Errno 2] No such file or directory: 'C:\\Users\\nikita.khatwani\\Documents\\UCSB\\District sytem - Engineering model\\District system - Engineering model'
/home/jupyter-nikitakhatwani/UCSB_Engineering model


In [15]:
import os

# Specify the correct directory path
directory_path = r"C:\Users\nikita.khatwani\Documents\UCSB\District system - Engineering model"

# Check if the directory exists
if os.path.exists(directory_path):
    os.chdir(directory_path)  # Change to the specified directory
    print("Changed directory successfully.")
else:
    print("Directory does not exist:", directory_path)


Directory does not exist: C:\Users\nikita.khatwani\Documents\UCSB\District system - Engineering model
